In [1]:
from railjson_generator import InfraBuilder, Location, SimulationBuilder
from railjson_generator.schema.infra.direction import Direction


In [2]:
builder = InfraBuilder()
tracks = [builder.add_track_section(length=1000) for _ in range(7)]


In [3]:
switch_0 = builder.add_point_switch(tracks[2].begin(), tracks[1].end(), tracks[0].end())
switch_1 = builder.add_point_switch(tracks[2].end(), tracks[3].begin(), tracks[4].begin())
switch_2 = builder.add_point_switch(tracks[4].end(), tracks[5].begin(), tracks[6].begin())


In [4]:
# Add detector and signals
for i in (2, 3, 4, 5, 6):
    track = tracks[i]
    detector = track.add_detector(position=200)
    signal = track.add_signal(detector.position - 25, Direction.START_TO_STOP, is_route_delimiter=True)
    signal.add_logical_signal("BAL", settings={"Nf": "true"})
    signal = track.add_signal(detector.position + 25, Direction.STOP_TO_START, is_route_delimiter=True)
    signal.add_logical_signal("BAL", settings={"Nf": "true"})

for i in (0, 1, 2, 4):
    track = tracks[i]
    detector = track.add_detector(position=800)
    signal = track.add_signal(detector.position - 25, Direction.START_TO_STOP, is_route_delimiter=True)
    signal.add_logical_signal("BAL", settings={"Nf": "true"})
    signal = track.add_signal(detector.position + 25, Direction.STOP_TO_START, is_route_delimiter=True)
    signal.add_logical_signal("BAL", settings={"Nf": "true"})


# Add operational points
my_op = builder.add_operational_point("my-op")
my_op.add_part(tracks[0], 500)
my_op.add_part(tracks[1], 500)

# Build infra: Generate BufferStops, TVDSections and Routes
infra = builder.build()

In [5]:
infra.save('infra.json')

In [6]:
from rlway.pyosrd import OSRD

In [7]:
sim = OSRD()

In [8]:
sim.route_limits

{'detector.5': ('track.0', 800.0),
 'detector.6': ('track.1', 800.0),
 'detector.0': ('track.2', 200.0),
 'detector.7': ('track.2', 800.0),
 'detector.1': ('track.3', 200.0),
 'detector.2': ('track.4', 200.0),
 'detector.8': ('track.4', 800.0),
 'detector.3': ('track.5', 200.0),
 'detector.4': ('track.6', 200.0),
 'buffer_stop.0': ('track.0', 0.0),
 'buffer_stop.1': ('track.1', 0.0),
 'buffer_stop.2': ('track.3', 1000.0),
 'buffer_stop.3': ('track.5', 1000.0),
 'buffer_stop.4': ('track.6', 1000.0)}

In [9]:
sim.draw_infra()

In [15]:
builder = SimulationBuilder()

first_train = builder.add_train_schedule(
    Location(tracks[0], 200), Location(tracks[5], 900), label="First", departure_time=10
)
first_train.add_stop(10, location=Location(tracks[4], 250))
first_train.add_stop(10, position=2550)
first_train.add_standard_single_value_allowance("percentage", 5, "MARECO")



In [16]:
sim_builder = builder.build()

In [17]:
sim_builder.save('simulation.json')

In [18]:
sim.run()

[11:32:58,952] [INFO]  [StandaloneSimulationCommand] Loading infra: ./infra.json
[11:32:58,991] [INFO]              [FullInfra] parsing infra
[11:32:59,061] [INFO]              [FullInfra] adaptation to kotlin
[11:32:59,095] [INFO]              [FullInfra] loading signals
[11:32:59,101] [INFO]              [FullInfra] building blocks
[11:32:59,107] [INFO]  [StandaloneSimulationCommand] Loading input: ./simulation.json
[11:32:59,118] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.1
[11:32:59,177] [DEBUG]             [Allowance] computing range n°1
[11:32:59,180] [DEBUG]             [Allowance]   computing section n°1
[11:32:59,180] [DEBUG]             [Allowance]   target time = 139.28565
[11:32:59,180] [DEBUG]             [Allowance]     starting attempt 1
[11:32:59,182] [DEBUG]             [Allowance]     envelope time 144.956
[11:32:59,182] [DEBUG]             [Allowance]     starting attempt 2
[11:32:59,182] [DEBUG]             [Allowance]     env

In [23]:
infra_builder = InfraBuilder()

T = [
    infra_builder.add_track_section(label="T"+str(id), length=10_000)
    for id in range(3)
]
T[0].add_buffer_stop(0, label="buffer_stop.0")

for i in [1, 2]:
    T[i].add_buffer_stop(T[i].length, label=f"buffer_stop.{i}")

infra_builder.add_point_switch(
    T[0].end(),
    T[1].begin(),
    T[2].begin(),
    label="DVG",
)
detectors = [T[0].add_detector(label="D0", position=T[0].length-180)]
detectors += [
    T[i].add_detector(
        label=f"D{i}",
        position=180,
    )
    for i in [1, 2]
]

In [24]:
signals = [
    T[0].add_signal(
        detectors[0].position-20,
        Direction.START_TO_STOP,
        is_route_delimiter=True,
        label="S0"
    ),
]
signals += [
    T[i].add_signal(
        detectors[i].position+20,
        Direction.STOP_TO_START,
        is_route_delimiter=True,
        label=f"S{i}"
    )
    for i in [1, 2]
]

for signal in signals:
        signal.add_logical_signal("BAL", settings={"Nf": "true"})


In [31]:
infra_builder.build().save("infra.json")

In [35]:
sim=OSRD()
sim.routes

['rt.buffer_stop.0->D0',
 'rt.D0->buffer_stop.1',
 'rt.D0->buffer_stop.2',
 'rt.buffer_stop.1->D1',
 'rt.D1->buffer_stop.0',
 'rt.buffer_stop.2->D2',
 'rt.D2->buffer_stop.0']

In [36]:
sim.route_tvds

{'rt.buffer_stop.0->D0': 'D0<->buffer_stop.0',
 'rt.D0->buffer_stop.1': 'DVG',
 'rt.D0->buffer_stop.2': 'DVG',
 'rt.buffer_stop.1->D1': 'D1<->buffer_stop.1',
 'rt.D1->buffer_stop.0': 'DVG',
 'rt.buffer_stop.2->D2': 'D2<->buffer_stop.2',
 'rt.D2->buffer_stop.0': 'DVG'}

In [41]:
sim_builder = SimulationBuilder()

sim_builder.add_train_schedule(
    Location(T[0], 50.),
    Location(T[1], T[1].length-50),
    label='train0',
    departure_time=0,
).add_standard_single_value_allowance(
    "percentage", 5, "MARECO"
)

sim_builder.add_train_schedule(
    Location(T[2], T[1].length-50.),
    Location(T[0], 50),
    label='train1',
    departure_time=100.,
)

built_simulation = sim_builder.build()

In [42]:
built_simulation.save('simulation.json')

In [43]:
sim = OSRD()
sim.run()

[11:48:52,690] [INFO]  [StandaloneSimulationCommand] Loading infra: ./infra.json
[11:48:52,719] [INFO]              [FullInfra] parsing infra
[11:48:52,779] [INFO]              [FullInfra] adaptation to kotlin
[11:48:52,808] [INFO]              [FullInfra] loading signals
[11:48:52,814] [INFO]              [FullInfra] building blocks
[11:48:52,819] [INFO]  [StandaloneSimulationCommand] Loading input: ./simulation.json
[11:48:52,830] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.4
[11:48:52,877] [DEBUG]             [Allowance] computing range n°1
[11:48:52,879] [DEBUG]             [Allowance]   computing section n°1
[11:48:52,880] [DEBUG]             [Allowance]   target time = 471.66945
[11:48:52,880] [DEBUG]             [Allowance]     starting attempt 1
[11:48:52,882] [DEBUG]             [Allowance]     envelope time 481.584
[11:48:52,882] [DEBUG]             [Allowance]     starting attempt 2
[11:48:52,882] [DEBUG]             [Allowance]     env